# Dynamic Environment

## Setup Environment

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [ ]:
# Agent config
capacity_percentage = 0.3
agent_radius = 0.2

# Map config
map_type = "swarm" # Available maps: empty, corridor, obstacle, cross, swarm
num_samples = 100 # Available num_samples: 100, 200, 300, 400, 500

# config starts and goals

start_regions = [
    Polygon([[0, 40], [0, 0], [50, 0], [50, 40]]),
    Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
]
# 
goal_regions = [
    Polygon([[160, 160], [160, 110], [200, 110], [200, 160]]),
    Polygon([[160, 40], [160, 0], [200, 0], [200, 40]]),
]

num_starts = 12
num_goals = 12


In [ ]:
# Load map and instance
import os
import pickle
from collections import Counter

import numpy as np 

map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f) 

node_capacities = [g_node.get_capacity(agent_radius) for g_node in gaussian_prm.gaussian_nodes]
print("average node capacity: ", np.average(node_capacities))

# Randomly choose starts and goals from the candidates, and assign them a weight
# Make sure all agents can fit in the start and goal configuration

starts_idx_set = gaussian_prm.get_node_index(start_regions)
goals_idx_set = gaussian_prm.get_node_index(goal_regions)

starts_idx = np.random.choice(starts_idx_set, num_starts, replace=False).tolist()
goals_idx = np.random.choice(goals_idx_set, num_goals, replace=False).tolist()

# Create weight pool

starts_pool = []
goals_pool = []

for start_idx in starts_idx:
    starts_pool += [start_idx] * gaussian_prm.gaussian_nodes[start_idx].get_capacity(agent_radius)

for goal_idx in goals_idx:
    goals_pool += [goal_idx] * gaussian_prm.gaussian_nodes[goal_idx].get_capacity(agent_radius)

# Decide agent based on capacity

instance_capacity = min(len(starts_pool), len(goals_pool))
num_agents = int(instance_capacity * capacity_percentage)

# Distribute agents to goals
start_per_agent = np.random.choice(starts_pool, num_agents, replace=False).tolist()
goal_per_agent = np.random.choice(goals_pool, num_agents, replace=False).tolist()

start_counts = Counter(start_per_agent)
goal_counts = Counter(goal_per_agent)

starts_idx = []
starts_agent_count = []
goals_idx = []
goals_agent_count = []

for start_idx, count in start_counts.items():
    starts_idx.append(start_idx)
    starts_agent_count.append(count)

for goal_idx, count in goal_counts.items():
    goals_idx.append(goal_idx)
    goals_agent_count.append(count)

print(starts_idx, goals_idx)
print(starts_agent_count, goals_agent_count)

In [ ]:
# Visualize instance
fig, ax = gaussian_prm.visualize_map()

for start in starts_idx:
    gaussian_prm.gaussian_nodes[start].visualize(ax)

for goal in goals_idx :
    gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor="b")

plt.show() 

## Add Dynamic Obstacles

We reserve the trajectory aggressively. If the nodes will be swept by the obstacle, 
we block the entry for agents during that timestep completely.

In [ ]:
from collections import defaultdict
from shapely.geometry import Point, LineString

num_dynamic_obstacle = 4 # TODO: add multiple obstacles
obstacle_radius = 10
obstacle_trajectory_1 = [(25, 80), (25, 60), (25, 50), (50, 50), (50, 60), (50, 50), (50, 60), 
                       (50, 50), (50, 60),(50, 50), (50, 60),
                       (50, 80), (50, 80), (75, 80), (100, 80), 
                       (50, 80), (50, 80), (75, 80), (100, 80), 
                       (125, 80), (125, 80), (125, 80), (125, 80),(150, 80), (175, 80), (160, 80)]

obstacle_trajectory_2 = [
    (60, 20), (60, 40), (60, 60), (60, 80),
    (75, 80), (75, 100), (75, 120), (75, 100), (75, 80), (75, 60), (75, 80),
    (75, 100), (74, 120),
    (95, 120), (125, 120),
    (125, 100), (125, 80), (125, 60), (125, 50), (100, 50),
    (80, 50), (80, 40), (80, 20)
    ]

obstacle_trajectories = [obstacle_trajectory_1, obstacle_trajectory_2]
# print(len(obstacle_trajectory))
# per-timestep swept-area
obstacle_geoms = []
for obstacle_trajectory in obstacle_trajectories:
    obstacle_geom = {}
    for t, (prev, next) in enumerate(zip(obstacle_trajectory[:-1], obstacle_trajectory[1:])):
        obstacle_geom[t] = [LineString((prev, next)).buffer(obstacle_radius)] # TODO: add more obstacles
    obstacle_geoms.append(obstacle_geom)

# per-timestep obstruct nodes. Modeled as capacity constraint.
occupancy_sets = []
obstructed_nodes = defaultdict(list)
for obstacle_geom in obstacle_geoms:
    occupancy_set = set()
    for t in range(len(obstacle_geom)):
        node_indices = gaussian_prm.get_overlapping_index(obstacle_geom[t])
        for node_idx in node_indices:
            occupancy_set.add((node_idx, t))
            obstructed_nodes[t].append(node_idx)
    occupancy_sets.append(occupancy_set)

obstacle_goal_dicts = []
# goal state obstruct nodes. Modeled as capacity constraint.as_integer_ratio
for obstacle_trajectory in obstacle_trajectories:
    node_indices = gaussian_prm.get_overlapping_index([Point(obstacle_trajectory[-1]).buffer(obstacle_radius)])
    obstacle_goal_dict = {node_idx: len(obstacle_trajectory) for node_idx in node_indices}
    obstacle_goal_dicts.append(obstacle_goal_dict)

max_timestep = max([len(trajectory) for trajectory in obstacle_trajectories])

In [ ]:
from matplotlib.patches import Circle

config = {
    "edgecolor": "lightgray"
}
fig, ax = gaussian_prm.visualize_g_nodes(**config)
# Plot one timestep
timestep = 12
steps = 5

obs_1_start = np.array(obstacle_trajectory_1[timestep])
obs_1_goal = np.array(obstacle_trajectory_1[timestep+1])

obs_2_start = np.array(obstacle_trajectory_2[timestep])
obs_2_goal = np.array(obstacle_trajectory_2[timestep+1])

alpha_step = np.linspace(0.1, 1, steps)

obs_1_traj = np.linspace(obs_1_start, obs_1_goal, steps)
obs_2_traj = np.linspace(obs_2_start, obs_2_goal, steps)

# Plot obstacle trajectory
node_indices = obstructed_nodes[timestep]
node_patches = [gaussian_prm.gaussian_nodes[idx].visualize(ax, edgecolor="r") for idx in node_indices]
for step in range(steps):
    ax.add_patch(Circle(obs_1_traj[step], radius=obstacle_radius, color="gray", fill=True, alpha=alpha_step[step]))
    ax.add_patch(Circle(obs_2_traj[step], radius=obstacle_radius, color="gray", fill=True, alpha=alpha_step[step]))
plt.show()
fig.savefig("./solutions/dynamic.pdf", bbox_inches="tight")

In [ ]:
# Instance Visualization
from matplotlib.patches import Circle
from matplotlib import animation

fig, ax = gaussian_prm.visualize_g_nodes(**config)
interpolated_points = 20

interpolated_trajectories = []
for obstacle_trajectory in obstacle_trajectories:
    interpolated_trajectory = []
    # interpolate between points
    for (prev, next) in zip(obstacle_trajectory[:-1], obstacle_trajectory[1:]):
        prev = np.array(prev)
        next = np.array(next)
        interpolated_trajectory.append(np.linspace(prev, next, interpolated_points))
    interpolated_trajectory = np.vstack(interpolated_trajectory)
    interpolated_trajectories.append(interpolated_trajectory)

frames = max([len(trajectory) for trajectory in interpolated_trajectories])

# Animation
obstacles = []
for obstacle_trajectory in obstacle_trajectories:
    obstacle = ax.add_patch(Circle(obstacle_trajectory[0], radius=obstacle_radius, color="grey", fill=True))
    obstacles.append(obstacle)
node_patches = []

def update(frame):
    global node_patches

    for patch in node_patches:
        patch.remove()
    node_patches = []

    # plot obstacle
    for i, obstacle in enumerate(obstacles):
        if len(interpolated_trajectories[i]) > frame:
            obstacle.set_center(interpolated_trajectories[i][frame])
        else:
            obstacle.set_center(interpolated_trajectories[i][-1]) # Stop at goal


    # color 
    node_indices = obstructed_nodes[frame//interpolated_points]
    node_patches = [gaussian_prm.gaussian_nodes[idx].visualize(ax, edgecolor="y") for idx in node_indices]

    return obstacles + node_patches

ani = animation.FuncAnimation(fig, update, frames=frames,
                              blit=True, interval=100)
ani.save("solutions/dynamic_environment.mp4", writer='ffmpeg', fps=24)
plt.close()


from IPython.display import Video
Video(filename="solutions/dynamic_environment.mp4", embed=True)



## Planner

In [ ]:
from st_gaussian_prm.solvers.macro import TEGSolver


solver = TEGSolver(gaussian_prm, agent_radius, 
              starts_agent_count=starts_agent_count, goals_agent_count=goals_agent_count,
              starts_idx=starts_idx, goals_idx=goals_idx,
              num_agents=num_agents, time_limit=180)

solution = solver.solve(occupancy_sets=occupancy_sets, obstacle_goal_dicts=obstacle_goal_dicts, max_timestep=len(obstacle_trajectory))
assert solution["success"], "solver failed."
timestep = solution["timestep"]
paths = solution["paths"]
g_nodes = solution["g_nodes"]
candid_starts_idx = solution["starts_idx"]
candid_goals_idx = solution["goals_idx"]

num_violation, max_violation_percentage = solver.eval_capacity(solution["paths"])
print("num violation: ", num_violation)
print("max violation percentage: ", max_violation_percentage)
print(solution["timestep"])


## Visualization

In [ ]:
from matplotlib import pyplot as plt
from st_gaussian_prm.utils import paths_to_macro
from st_gaussian_prm.solvers.micro import EvaluationSolver

agent_radius = 0.08 # to guarantee that we have enough samples in each node for visualization

macro_solution = paths_to_macro(paths)

timestep = max(macro_solution.keys())

solver = EvaluationSolver(g_nodes, macro_solution, agent_radius, timestep, 
                                     num_agents, 
                                     candid_starts_idx,
                                     candid_goals_idx,
                                     starts_agent_count,
                                     goals_agent_count
                                    )

single_agent_paths, cost = solver.solve()
fig, ax = gaussian_prm.visualize_map()

cmap = plt.get_cmap("rainbow")
colors = [cmap(i / num_agents) for i in range(num_agents)]

for i, path in enumerate(single_agent_paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color=colors[i], linewidth=0.8, alpha=0.5)

for start in starts_idx:
    gaussian_prm.gaussian_nodes[start].visualize(ax)

for goal in goals_idx:
    gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor="b")

# Plot one timestep
timestep = 14
steps = 5

obs_1_start = np.array(obstacle_trajectory_1[timestep])
obs_1_goal = np.array(obstacle_trajectory_1[timestep+1])

obs_2_start = np.array(obstacle_trajectory_2[timestep])
obs_2_goal = np.array(obstacle_trajectory_2[timestep+1])

alpha_step = np.linspace(0.1, 1, steps)

obs_1_traj = np.linspace(obs_1_start, obs_1_goal, steps)
obs_2_traj = np.linspace(obs_2_start, obs_2_goal, steps)

# Plot obstacle trajectory
node_indices = obstructed_nodes[timestep]
node_patches = [gaussian_prm.gaussian_nodes[idx].visualize(ax, edgecolor="r") for idx in node_indices]
for step in range(steps):
    ax.add_patch(Circle(obs_1_traj[step], radius=obstacle_radius, color="gray", fill=True, alpha=alpha_step[step], zorder=10))
    ax.add_patch(Circle(obs_2_traj[step], radius=obstacle_radius, color="gray", fill=True, alpha=alpha_step[step], zorder=10))

plt.show()
fig.savefig("./solutions/dynamic_solution.pdf", bbox_inches="tight")

In [ ]:
# Animate path
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle

from IPython.display import Video

# Add interpolations 
import numpy as np

def add_linear_interpolation_points(path, subdiv=5):
    """
    Insert linearly-interpolated positions between consecutive points.
    
    Parameters
    ----------
    path : array-like of shape (N, 2)
        The original list/array of N 2D points, e.g. [(x1,y1), (x2,y2), ...].
    subdiv : int
        Number of intervals to subdivide each segment. 
        For each original segment, we will add 'subdiv - 1' new interior points 
        (except that we skip duplicates at the shared boundary).

    Returns
    -------
    new_path : np.ndarray of shape (M, 2)
        The new path including the original points and the added interpolation points.
    """
    path = np.asarray(path)
    new_path = []

    # Iterate over pairs of consecutive points
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        
        # Create subdiv+1 points in [start, end] using linspace for each dimension
        xs = np.linspace(start[0], end[0], subdiv + 1)
        ys = np.linspace(start[1], end[1], subdiv + 1)
        
        # Add each intermediate point except the very last one
        # to avoid duplicating the next segment's start
        for j in range(subdiv):
            new_path.append([xs[j], ys[j]])

    # Finally add the very last point of the last segment
    new_path.append(path[-1].tolist())
    
    return np.array(new_path)

interpolated_paths = []
for p in single_agent_paths:
    new_path = add_linear_interpolation_points(p, interpolated_points)
    interpolated_paths.append(new_path)

# Interpolate obstacle trajectory
interpolated_trajectories = []
for obstacle_trajectory in obstacle_trajectories: 
    interpolated_trajectory = []
    for (prev, next) in zip(obstacle_trajectory[:-1], obstacle_trajectory[1:]):
        prev = np.array(prev)
        next = np.array(next)
        interpolated_trajectory.append(np.linspace(prev, next, interpolated_points))

    interpolated_trajectory = np.vstack(interpolated_trajectory)
    interpolated_trajectories.append(interpolated_trajectory)

def animate_solution(agent_radius, paths, obstacle_trajectories, fig, ax, fig_path="."):
    """
        Visualize solution trajectory provided instance
    """
    agents = []
    cmap = plt.get_cmap("tab10")
    
    for i in range(len(paths)):
        loc = paths[i][0]
        circle = Circle((loc[0], loc[1]), radius=agent_radius, color=cmap(i % 10))
        agents.append(circle)
        ax.add_patch(circle)
    
    obstacles = []
    for obstacle_trajectory in obstacle_trajectories:
        obstacle = ax.add_patch(Circle(obstacle_trajectory[0], radius=obstacle_radius, color="grey", fill=True))
        obstacles.append(obstacle)

    def init():
        return agents

    def update(frame):
        for i, obstacle in enumerate(obstacles):
            if frame < len(obstacle_trajectories[i]):
                obstacle.set_center(obstacle_trajectories[i][frame])
            else:
                obstacle.set_center(obstacle_trajectories[i][-1])

        for agent, traj in zip(agents, paths):
            agent.set_center(traj[frame])
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0]), 
                         init_func=init, blit=True, interval=100)
    anim.save(f"{fig_path}/apf_solution.mp4", writer='ffmpeg', fps=24)
    plt.close()

fig_path = "solutions"
fig, ax = gaussian_prm.visualize_map()
animate_solution(agent_radius, interpolated_paths, interpolated_trajectories, fig, ax, fig_path=fig_path)
Video(filename="solutions/apf_solution.mp4", embed=True)